In [1]:
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [2]:
n_vocab = 6785
n_embed = 300
n_filters = 250
n_classes = 2

In [3]:
device = torch.device("cuda")
device

device(type='cuda')

In [4]:
class Model(nn.Module):
    '''Deep Pyramid Convolutional Neural Networks for Text Categorization'''
    def __init__(self, n_vocab, n_embed, n_filters, n_classes):
        super(Model, self).__init__()
        
        self.embedding = nn.Embedding(n_vocab, n_embed, padding_idx=n_vocab - 1)
        self.conv_region = nn.Conv2d(1, n_filters, (3, n_embed), stride=1)
        
        self.conv = nn.Conv2d(n_filters, n_filters, (3, 1), stride=1)
        self.max_pool = nn.MaxPool2d(kernel_size=(3, 1), stride=2)
        self.padding1 = nn.ZeroPad2d((0, 0, 1, 1))  # top bottom
        self.padding2 = nn.ZeroPad2d((0, 0, 0, 1))  # bottom
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(n_filters, n_classes)

    def forward(self, x):
        print(x[0].size())
        x = x[0]
        x = self.embedding(x)
        x = x.unsqueeze(1)         # [batch_size, 250, seq_len, 1]
        # x = self.conv_region(x)  # [batch_size, 250, seq_len-3+1, 1]
        px = self.conv_region(x)
        print("px:", px.size())
#         px = self.dropout(px)
#         print("px:", px.size())
        x = self.padding1(px)
        
        # x = self.padding1(x)     # [batch_size, 250, seq_len, 1]
        x = self.relu(x)
        x = self.conv(x)           # [batch_size, 250, seq_len-3+1, 1]
        x = self.padding1(x)       # [batch_size, 250, seq_len, 1]
        x = self.relu(x)
        x = self.conv(x)           # [batch_size, 250, seq_len-3+1, 1]
        print("0----", x.size())
        x = x + px
#         for _ in range(4):
        while x.size()[2] >= 2:
            x = self._block(x)
            print(x.size())
        print("1----", x.size())
        x = x.squeeze()          # [batch_size, n_filters(250)]
        print("2----", x.size())
        x = self.dropout(x)
        x = self.fc(x)
        print("3----", x.size())
        return x

    def _block(self, x):
        x = self.padding2(x)
        px = self.max_pool(x)

        x = self.padding1(px)
        x = F.relu(x)
        x = self.conv(x)

        x = self.padding1(x)
        x = F.relu(x)
        x = self.conv(x)

        x = x + px
        return x

In [5]:
model = Model(n_vocab, n_embed, n_filters, n_classes)

In [6]:
model.to(device)

Model(
  (embedding): Embedding(6785, 300, padding_idx=6784)
  (conv_region): Conv2d(1, 250, kernel_size=(3, 300), stride=(1, 1))
  (conv): Conv2d(250, 250, kernel_size=(3, 1), stride=(1, 1))
  (max_pool): MaxPool2d(kernel_size=(3, 1), stride=2, padding=0, dilation=1, ceil_mode=False)
  (padding1): ZeroPad2d(padding=(0, 0, 1, 1), value=0.0)
  (padding2): ZeroPad2d(padding=(0, 0, 0, 1), value=0.0)
  (relu): ReLU()
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=250, out_features=2, bias=True)
)

In [7]:
#(x, seq_len), y
# x = torch.LongTensor([_[0] for _ in datas]).to(self.device)
# y = torch.LongTensor([_[1] for _ in datas]).to(self.device)
# # pad前的长度(超过pad_size的设为pad_size)
# seq_len = torch.LongTensor([_[2] for _ in datas]).to(self.device)
# return (x, seq_len), y
pad_size = 1500

In [8]:
data = []
for i in range(128):
    x = torch.randint(0, n_vocab, (pad_size,))
    y = 1
    seq_len = pad_size
    data.append((x, y, seq_len))

In [9]:
x = torch.LongTensor([_[0].numpy() for _ in data]).to(device)
y = torch.LongTensor([_[1] for _ in data]).to(device)
seq_len = torch.LongTensor([_[2] for _ in data]).to(device)
# batch = (x, seq_len), y

In [10]:
new_data = (x, seq_len), y

In [11]:
new_data

((tensor([[4735, 2316, 5815,  ..., 5861, 1957, 6676],
          [ 338, 3356, 4412,  ..., 2147, 6649, 3719],
          [5932, 1601, 2034,  ..., 2308,  289, 4426],
          ...,
          [1819, 2558, 4822,  ..., 4615, 2433, 5891],
          [2806,  269, 4575,  ...,  725, 1531, 4668],
          [3651, 3460,  480,  ..., 6723, 2548,  534]], device='cuda:0'),
  tensor([1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
          1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
          1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
          1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
          1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
          1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
          1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
          1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500

In [12]:
new_data[0][0].size()

torch.Size([128, 1500])

In [13]:
model(new_data[0])
# 1---- torch.Size([128, 250, 2, 1])
# 2---- torch.Size([128, 250, 2])
## 正常
# 1---- torch.Size([128, 250, 1, 1])
# 2---- torch.Size([128, 250])
# 3---- torch.Size([128, 2])

torch.Size([128, 1500])
px: torch.Size([128, 250, 1498, 1])
px: torch.Size([128, 250, 1498, 1])
0---- torch.Size([128, 250, 1498, 1])
torch.Size([128, 250, 749, 1])
torch.Size([128, 250, 374, 1])
torch.Size([128, 250, 187, 1])
torch.Size([128, 250, 93, 1])
torch.Size([128, 250, 46, 1])
torch.Size([128, 250, 23, 1])
torch.Size([128, 250, 11, 1])
torch.Size([128, 250, 5, 1])
torch.Size([128, 250, 2, 1])
torch.Size([128, 250, 1, 1])
1---- torch.Size([128, 250, 1, 1])
2---- torch.Size([128, 250])
3---- torch.Size([128, 2])


tensor([[-1.9882,  1.3456],
        [-1.6952,  7.4155],
        [ 0.9635,  4.5713],
        [-2.8678,  6.6539],
        [ 2.9253,  2.9582],
        [ 0.6886,  7.5188],
        [-1.8106,  7.1478],
        [ 1.2834,  6.5948],
        [-2.6055,  3.4480],
        [-1.3335,  5.8247],
        [-0.3499,  6.3552],
        [-1.8354,  4.9861],
        [-0.9082,  4.3563],
        [ 0.0739,  5.2568],
        [-0.7252,  5.6073],
        [ 0.6507,  7.0877],
        [-0.4518,  5.6180],
        [-2.7945,  4.0830],
        [-0.6449,  6.3809],
        [-2.2850,  4.4531],
        [-1.4923,  7.1569],
        [-0.2675,  4.9332],
        [ 1.2943,  5.1248],
        [-0.0620,  5.9816],
        [ 1.0221,  2.5349],
        [ 0.2962,  6.2708],
        [ 1.8221,  7.6633],
        [ 2.6781,  6.2605],
        [-0.3710,  5.9253],
        [-1.2596,  6.6335],
        [ 0.2712,  7.3309],
        [ 1.1502,  7.3616],
        [ 3.7757,  4.9064],
        [ 0.6839,  6.3405],
        [-0.5159,  8.9584],
        [-1.0605,  6

In [1]:
new_data[0][0].size()

NameError: name 'new_data' is not defined

In [16]:
nn.ZeroPad2d((0, 0, 1, 1))

ZeroPad2d(padding=(0, 0, 1, 1), value=0.0)

In [17]:
embedding = nn.Embedding(n_vocab, n_embed, padding_idx=n_vocab - 1)

In [18]:
x = new_data[0][0]

In [19]:
x.size()

torch.Size([128, 1500])

In [20]:
x = embedding(x)
x.size()

torch.Size([128, 1500, 300])

In [22]:
region_conv = nn.Conv2d(1, n_filters, (3, n_embed), stride=1)

In [23]:
x = x.unsqueeze(1)
x.size()

torch.Size([128, 1, 1500, 300])

In [24]:
x = region_conv(x)
x.size()

torch.Size([128, 250, 1498, 1])

In [25]:
padding1 = nn.ZeroPad2d((0, 0, 1, 1))

In [26]:
x = padding1(x)
x.size()

torch.Size([128, 250, 1500, 1])

In [27]:
conv = nn.Conv2d(n_filters, n_filters, (3, 1), stride=1)

In [28]:
relu = nn.ReLU()

In [29]:
x = relu(x)
print(x.size())
x = conv(x)
print(x.size())

torch.Size([128, 250, 1500, 1])
torch.Size([128, 250, 1498, 1])


In [30]:
x = padding1(x)
print(x.size())
x = relu(x)
print(x.size())
x = conv(x)
print(x.size())

torch.Size([128, 250, 1500, 1])
torch.Size([128, 250, 1500, 1])
torch.Size([128, 250, 1498, 1])


In [31]:
padding2 = nn.ZeroPad2d((0, 0, 0, 1))

In [32]:
maxpool = nn.MaxPool2d(kernel_size=(3, 1), stride=2)

In [33]:
x = padding2(x)

In [34]:
print(x.size())

torch.Size([128, 250, 1499, 1])


In [35]:
px = maxpool(x)
print(px.size())

torch.Size([128, 250, 749, 1])


In [36]:
x = padding1(px)
print(x.size())
x = relu(x)
print(x.size())
x = conv(x)
print(x.size())

torch.Size([128, 250, 751, 1])
torch.Size([128, 250, 751, 1])
torch.Size([128, 250, 749, 1])


In [37]:
x = padding1(x)
print(x.size())
x = relu(x)
print(x.size())
x = conv(x)
print(x.size())

torch.Size([128, 250, 751, 1])
torch.Size([128, 250, 751, 1])
torch.Size([128, 250, 749, 1])


In [38]:
x = x + px
x.size()

torch.Size([128, 250, 749, 1])

In [48]:
m = nn.Dropout2d(p=0.2)
input = torch.randn(20, 16, 32, 32)
output = m(input)
output.size()

torch.Size([20, 16, 32, 32])

In [49]:
m = nn.Dropout(p=0.2)
input = torch.randn(20, 16)
output = m(input)
output.size()

torch.Size([20, 16])